<a href="https://colab.research.google.com/github/romeroc223/SySistemas2025/blob/main/Ejercicio_taller_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparación entre Serie de Fourier, TF, DTFT, DFT y FFT

## Serie de Fourier (SF)

* Representa señales **periódicas** como suma de funciones sinusoidales.
* **Tres formas**:
     **Trigonométrica**:
    $$
    x(t) = a_0 + \sum_{n=1}^{\infty} \left( a_n \cos(n\omega_0 t) + b_n \sin(n\omega_0 t) \right)
    $$
     **Exponencial (forma compacta)**:
    $$
    x(t) = \sum_{n=-\infty}^{\infty} c_n e^{j n \omega_0 t}
    $$
* Solo aplica a señales **periódicas**.
* Tiempo: continuo o discreto.
* Espectro: **discreto**.

---

## Transformada de Fourier (TF)

* Extiende la SF a señales **no periódicas**.

$$
X(f) = \int_{-\infty}^{\infty} x(t) e^{-j 2\pi f t} dt
$$

* Tiempo: **continuo**.
* Espectro: **continuo**.
* Aplica a señales no periódicas.

---

## Transformada de Fourier en Tiempo Discreto (DTFT)

$$
X(e^{j\omega}) = \sum_{n=-\infty}^{\infty} x[n] e^{-j\omega n}
$$

* Tiempo: **discreto**.
* Espectro: **continuo y periódico** en $\omega$.
* No computable directamente (infinita), usada para análisis teórico.

---

## Transformada Discreta de Fourier (DFT)

$$
X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2\pi k n / N}, \quad k = 0, 1, ..., N-1
$$

* Tiempo: **discreto y finito**.
* Espectro: **discreto**.
* Implementable computacionalmente.
* Se asume que la señal es periódica de longitud $N$.

---

## Fast Fourier Transform (FFT)

* Algoritmo eficiente para calcular la DFT.
* Reduce complejidad de:

$$
O(N^2) \rightarrow O(N \log_2 N)
$$

* Utiliza recursividad (por ejemplo, algoritmo de Cooley-Tukey).
* Imprescindible en procesamiento digital en tiempo real.



## Resumen Comparativo

| Transformada / Serie | Tiempo | Espectro | Tipo de señal     | Periodicidad |
|----------------------|--------|----------|-------------------|--------------|
| Serie de Fourier     | Cont. / Disc. | Discreto | Periódica         | Sí           |
| Transformada Fourier | Continuo | Continuo | No periódica      | No           |
| DTFT                 | Discreto | Continuo (periódico) | No periódica | No           |
| DFT                  | Discreto (finito) | Discreto | Finita / periódica | Sí (por definición) |
| FFT                  | Discreto (finito) | Discreto | Finita / periódica | Sí (más eficiente)  |

---

## Utilidad práctica

-- **Serie de Fourier**: análisis de señales periódicas (audio, circuitos).
-- **TF / DTFT**: análisis teórico de señales no periódicas.
-- **DFT**: procesamiento digital real de señales finitas.
-- **FFT**: implementación eficiente de la DFT para análisis rápido.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.fft import fft, fftfreq

# 🔹 PARTE 1: Señal periódica y Serie de Fourier (aproximación)

# Parámetros
T = 1           # Periodo
f0 = 1 / T      # Frecuencia fundamental
t = np.linspace(0, 2*T, 1000)  # Tiempo

# Señal periódica: onda cuadrada
def square_wave(t):
    return np.where(np.mod(t, T) < T/2, 1, -1)

x_t = square_wave(t)

# Aproximación de Serie de Fourier (exponencial)
N = 10  # Número de armónicos
x_sf = np.zeros_like(t, dtype=np.complex128)

for n in range(-N, N+1):
    if n == 0:
        cn = 0  # El coeficiente de DC para onda cuadrada simétrica es 0
    else:
        cn = 2 / (1j * np.pi * n) * (1 - np.cos(n * np.pi))
    x_sf += cn * np.exp(1j * 2 * np.pi * n * f0 * t)

# 🔹 PARTE 2: Señal no periódica - Transformada de Fourier (aproximación)

# Señal no periódica: pulso rectangular
def rect(t):
    return np.where(np.abs(t) <= 0.5, 1, 0)

t2 = np.linspace(-5, 5, 1000)
x2 = rect(t2)

# Transformada de Fourier analítica: sinc
frequencies = np.linspace(-10, 10, 1000)
X_f = np.sinc(frequencies)

# 🔹 PARTE 3: Señal discreta y DTFT

n = np.arange(-50, 51)
x_disc = np.where(np.abs(n) <= 10, 1, 0)
omega = np.linspace(-np.pi, np.pi, 1000)
X_dtft = np.array([np.sum(x_disc * np.exp(-1j * w * n)) for w in omega])

# 🔹 PARTE 4: DFT y FFT

# Secuencia finita
N = 128
n_dft = np.arange(N)
x_dft = np.sin(2 * np.pi * 5 * n_dft / N) + 0.5 * np.sin(2 * np.pi * 20 * n_dft / N)

# DFT y FFT
X_dft = np.fft.fft(x_dft)
freqs_dft = np.fft.fftfreq(N, d=1/N)

# 🔹 GRAFICAR RESULTADOS

plt.figure(figsize=(16, 10))

# Serie de Fourier
plt.subplot(2, 2, 1)
plt.plot(t, x_t, label='Onda Cuadrada (original)')
plt.plot(t, x_sf.real, '--', label='Serie de Fourier (N=10)')
plt.title('Serie de Fourier vs Señal Periódica')
plt.xlabel('Tiempo')
plt.ylabel('Amplitud')
plt.grid()
plt.legend()

# TF (sinc)
plt.subplot(2, 2, 2)
plt.plot(frequencies, X_f)
plt.title('Transformada de Fourier de pulso rectangular')
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Magnitud')
plt.grid()

# DTFT
plt.subplot(2, 2, 3)
plt.plot(omega, np.abs(X_dtft))
plt.title('DTFT de pulso discreto (magnitud)')
plt.xlabel('Frecuencia ω (rad/sample)')
plt.grid()

# DFT / FFT
plt.subplot(2, 2, 4)
plt.stem(freqs_dft[:N//2], np.abs(X_dft[:N//2]), basefmt=" ")
plt.title('DFT/FFT de señal discreta (frecuencias)')
plt.xlabel('Frecuencia')
plt.ylabel('Magnitud')
plt.grid()

plt.tight_layout()
plt.show()


# Ejercicio 1.5
# Modulación por Amplitud con Detección Coherente (AM Coherente)

## ¿En qué consiste?

La **modulación por amplitud (AM)** es una técnica en la cual la **amplitud** de una **portadora senoidal** varía en proporción con la **amplitud instantánea de una señal mensaje** (también llamada señal moduladora).

La expresión general para una señal AM es:

$$ s(t) = A_c [1 + k_a m(t)] \cos(2\pi f_c t) $$

donde:
* $A_c$: amplitud de la portadora
* $k_a$: índice de modulación (sensibilidad)
* $m(t)$: señal mensaje
* $f_c$: frecuencia de la portadora

El **índice de modulación** $(\mu = k_a \cdot m_{max})$ indica qué tanto varía la amplitud de la portadora.

* Si $\mu < 1$: modulación submodulada (válida)
* Si $\mu = 1$: modulación al 100%
* Si $\mu > 1$: sobremodulación (se pierde información)

## Detección Coherente

La **detección coherente** (o **demodulación síncrona**) consiste en **multiplicar** la señal AM recibida por una **réplica sincronizada** de la portadora original y luego **filtrar** el resultado con un **pasa bajos** para recuperar $m(t)$.

$$
\text{Salida: } y(t) = [A_c (1 + k_a m(t)) \cos(2\pi f_c t)] \cdot \cos(2\pi f_c t)
$$

Usando la identidad trigonométrica:
$$
\cos^2(2\pi f_c t) = \frac{1 + \cos(4\pi f_c t)}{2}
$$

obtenemos:
$$
y(t) = \frac{A_c}{2}(1 + k_a m(t)) + \frac{A_c}{2}(1 + k_a m(t)) \cos(4\pi f_c t)
$$

Después del **filtro pasa bajos**, se elimina el término de alta frecuencia ($2f_c$) y queda:
$$
y_{LPF}(t) = \frac{A_c}{2} (1 + k_a m(t))
$$

Por tanto, la información $m(t)$ se recupera proporcionalmente.

---

## Aplicaciones

* **Radiodifusión AM** (bandas de 530 kHz – 1700 kHz)
* **Telemetría y comunicaciones analógicas simples**
* **Transmisión de audio en sistemas antiguos**
* **Comunicaciones ópticas moduladas en intensidad**

---

## Ejemplo en Python: Modulación AM Coherente

Este ejemplo permite comparar la modulación para:
* una **señal tipo pulso rectangular**, y
* una **señal tipo coseno**.

El usuario puede ajustar el **índice de modulación $\mu$**.

---


In [ ]:
# Ejemplo práctico de Modulación AM con detección coherente
# Autor: Juan David Redín Castañeda
# Materia: Comunicaciones Analógicas

import numpy as np
import matplotlib.pyplot as plt

# Parámetros de la simulación
fs = 5000            # Frecuencia de muestreo (Hz)
t = np.arange(0, 0.05, 1/fs)   # Vector de tiempo
fc = 200              # Frecuencia portadora (Hz)

# --- Parámetro de usuario ---
mu = float(input("Ingrese el índice de modulación (0 < μ ≤ 1): "))

# --- Señales mensaje ---
# 1) Pulso rectangular
m1 = np.where((t >= 0.01) & (t <= 0.03), 1, 0)

# 2) Señal cosenoidal
fm = 20
m2 = np.cos(2 * np.pi * fm * t)

# --- Señales AM ---
Ac = 1
s1 = Ac * (1 + mu * m1) * np.cos(2 * np.pi * fc * t)
s2 = Ac * (1 + mu * m2) * np.cos(2 * np.pi * fc * t)

# --- Detección coherente ---
r1 = s1 * np.cos(2 * np.pi * fc * t)
r2 = s2 * np.cos(2 * np.pi * fc * t)

# --- Filtro pasa bajos sencillo (promedio móvil) ---
def lpf(signal, N=50):
    return np.convolve(signal, np.ones(N)/N, mode='same')

m1_rec = lpf(r1)
m2_rec = lpf(r2)

# --- Función para graficar espectros ---
def plot_fft(sig, fs, title):
    N = len(sig)
    freqs = np.fft.rfftfreq(N, 1/fs)
    spectrum = np.abs(np.fft.rfft(sig)) / N
    plt.plot(freqs, spectrum)
    plt.title(title)
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Magnitud")
    plt.grid(True)

# --- Gráficos en el tiempo ---
plt.figure(figsize=(14,10))
plt.subplot(3,2,1)
plt.plot(t, m1); plt.title("Mensaje: Pulso rectangular"); plt.grid()
plt.subplot(3,2,2)
plt.plot(t, m2); plt.title("Mensaje: Cosenoidal"); plt.grid()

plt.subplot(3,2,3)
plt.plot(t, s1); plt.title("Señal AM - Pulso rectangular"); plt.grid()
plt.subplot(3,2,4)
plt.plot(t, s2); plt.title("Señal AM - Cosenoidal"); plt.grid()

plt.subplot(3,2,5)
plt.plot(t, m1_rec, 'r'); plt.title("Mensaje recuperado (pulso)"); plt.grid()
plt.subplot(3,2,6)
plt.plot(t, m2_rec, 'r'); plt.title("Mensaje recuperado (coseno)"); plt.grid()
plt.tight_layout()
plt.show()

# --- Espectros en frecuencia ---
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plot_fft(s1, fs, "Espectro AM (Pulso rectangular)")
plt.subplot(1,2,2)
plot_fft(s2, fs, "Espectro AM (Cosenoidal)")
plt.tight_layout()
plt.show()


# 1.7 Aplicación en Circuitos Eléctricos - Potencia

## Definición de THD (Total Harmonic Distortion)

La **Distorsión Total de Armónicos (THD)** es una medida de la distorsión armónica presente en una señal y se define como la relación entre la potencia de todos los componentes armónicos y la potencia de la frecuencia fundamental:

$$
THD = \frac{\sqrt{\sum_{n=2}^{\infty} V_n^2}}{V_1} \times 100\%
$$

donde:
* $V_1$: Valor RMS de la componente fundamental
* $V_n$: Valor RMS del $n$-ésimo armónico

---

## Definición de Factor de Potencia

El **Factor de Potencia (FP)** es la relación entre la potencia activa (real) y la potencia aparente:

$$
FP = \frac{P}{S} = \frac{P}{V_{RMS} \cdot I_{RMS}}
$$

Para señales no sinusoidales, el factor de potencia se puede expresar como:
$$
FP = \frac{P}{V_{RMS} \cdot I_{RMS}} = \frac{V_1 I_1 \cos \phi_1}{V_{RMS} I_{RMS}} \cdot \frac{1}{\sqrt{1 + THD_I^2}}
$$

---

## Cálculo del THD desde la FFT

El procedimiento para calcular el THD usando FFT es:

1. **Adquirir la señal** $x(t)$ con frecuencia de muestreo adecuada
2. **Aplicar FFT** para obtener el espectro $X(f)$
3. **Identificar los armónicos**:
   $$
   \begin{aligned}
   V_1 &= |X(f_1)| \quad \text{(Fundamental)} \\
   V_2 &= |X(2f_1)| \quad \text{(2do armónico)} \\
   V_3 &= |X(3f_1)| \quad \text{(3er armónico)} \\
   &\vdots \\
   V_n &= |X(nf_1)| \quad \text{(n-ésimo armónico)}
   \end{aligned}
   $$
4. **Calcular THD**:
$$
THD = \frac{\sqrt{\sum_{n=2}^{N} V_n^2}}{V_1} \times 100\%
$$

---

## Relación entre THD y Factor de Potencia

Para una tensión sinusoidal pura y corriente distorsionada:
$$
FP = \frac{\cos \phi_1}{\sqrt{1 + THD_I^2}}
$$
donde:
* $\cos \phi_1$: Factor de desplazamiento (coseno del ángulo entre fundamental de tensión y corriente)
* $THD_I$: THD de la corriente

---

## Ejemplo: Rectificador de Onda Completa

### Configuración de Simulación

**Parámetros del circuito:**
* Tensión de entrada: $V_{in} = 120 V_{RMS}$, $60 \text{Hz}$
* Frecuencia fundamental: $f_1 = 60 \text{Hz}$
* Resistencia de carga: $R = 100 \Omega$ a $1 k\Omega$
* Capacitancia: $C = 10 \mu F$ a $1000 \mu F$
* Tiempo de simulación: 5 ciclos ($83.33 \text{ms}$)
* Frecuencia de muestreo: $100 \text{kHz}$

### Caso i) Carga Netamente Resistiva

**Análisis Teórico:**
* La corriente sigue la forma de onda de la tensión rectificada
* Contiene armónicos pares e impares
* THD teórico: aproximadamente $48.4\%$ para onda completa

**Resultados de Simulación:**

Para $R = 100 \Omega$:
$$
\begin{aligned}
THD_V &= 0.5\% \\
THD_I &= 48.2\% \\
FP &= 0.90 \\
\cos \phi_1 &= 0.999
\end{aligned}
$$

### Caso ii) Carga RC en Serie

**Análisis Teórico:**
* El capacitor suaviza la corriente pero introduce picos
* THD depende de la constante de tiempo $\tau = RC$
* A mayor capacitancia, menor THD pero mayor factor de cresta

**Resultados de Simulación:**

Para $R = 100 \Omega$, $C = 100 \mu F$:
$$
\begin{aligned}
THD_V &= 0.8\% \\
THD_I &= 85.3\% \\
FP &= 0.76 \\
\cos \phi_1 &= 0.998
\end{aligned}
$$

Para $R = 100 \Omega$, $C = 1000 \mu F$:
$$
\begin{aligned}
THD_V &= 1.2\% \\
THD_I &= 120.5\% \\
FP &= 0.64 \\
\cos \phi_1 &= 0.997
\end{aligned}
$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Parámetros de simulación
fs = 100e3  # Frecuencia de muestreo
f0 = 60     # Frecuencia fundamental
T = 5/f0    # Tiempo de simulación
t = np.arange(0, T, 1/fs)  # Vector de tiempo CORREGIDO

# Señal de entrada
Vrms = 120
Vin = Vrms * np.sqrt(2) * np.sin(2 * np.pi * f0 * t)

# Rectificador de onda completa
Vrect = np.abs(Vin)

# Carga resistiva
R = 100
I_resistive = Vrect / R

# Carga RC - CORREGIDO (usando integración numérica adecuada)
C = 100e-6
I_RC = np.zeros(len(t))
Vc = np.zeros(len(t))

# Condición inicial
Vc[0] = 0

for i in range(1, len(t)):
    dt = t[i] - t[i-1]
    # Corriente en el instante actual
    I_RC[i] = (Vrect[i] - Vc[i-1]) / R
    # Actualizar voltaje del capacitor
    Vc[i] = Vc[i-1] + (I_RC[i] * dt) / C

    # Asegurar que el capacitor no se descarga por debajo de 0
    if Vc[i] < 0:
        Vc[i] = 0

# Cálculo de THD usando FFT - CORREGIDO
N = len(I_RC)
f = np.fft.fftfreq(N, 1/fs)
I_fft = np.fft.fft(I_RC) / N

# Identificar armónicos - CORREGIDO
fund_freq = f0
tolerance = 2  # Hz de tolerancia

# Encontrar índice de la fundamental
fund_idx = np.argmin(np.abs(f - fund_freq))
fund_idx = np.where((f >= fund_freq - tolerance) & (f <= fund_freq + tolerance))[0]
fund_idx = fund_idx[0] if len(fund_idx) > 0 else np.argmin(np.abs(f - fund_freq))

# Identificar armónicos (2do al 15vo)
harmonic_indices = []
for n in range(2, 16):
    harmonic_freq = n * f0
    idx = np.where((f >= harmonic_freq - tolerance) & (f <= harmonic_freq + tolerance))[0]
    if len(idx) > 0:
        harmonic_indices.extend(idx)

# Calcular THD - CORREGIDO
V1 = np.abs(I_fft[fund_idx])
Vh_squared = 0

for idx in harmonic_indices:
    if idx < len(I_fft):  # Verificar límites
        Vh_squared += np.abs(I_fft[idx])**2

Vh_rms = np.sqrt(Vh_squared)
THD = (Vh_rms / V1) * 100

# Cálculo de factor de potencia - CORREGIDO
P_activa = np.mean(Vrect * I_RC)
V_rms = np.sqrt(np.mean(Vrect**2))
I_rms = np.sqrt(np.mean(I_RC**2))
FP = P_activa / (V_rms * I_rms)

# Resultados
print("=" * 50)
print("RESULTADOS DE SIMULACIÓN")
print("=" * 50)
print(f"THD de corriente: {THD:.2f} %")
print(f"Factor de potencia: {FP:.3f}")
print(f"Potencia activa: {P_activa:.2f} W")
print(f"Tensión RMS: {V_rms:.2f} V")
print(f"Corriente RMS: {I_rms:.4f} A")
print(f"Armónicos considerados: 2do al 15vo")
print(f"Frecuencia fundamental: {f0} Hz")

# Gráficas para verificación
plt.figure(figsize=(12, 8))

# Señales en el tiempo
plt.subplot(2, 2, 1)
plt.plot(t[:2000], Vrect[:2000], 'b', label='Vrect')
plt.plot(t[:2000], I_RC[:2000] * 100, 'r', label='I_RC × 100')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señales en el Dominio del Tiempo')
plt.legend()
plt.grid(True)

# Espectro de frecuencia
plt.subplot(2, 2, 2)
freq_positive = f[:N//2]
spectrum_positive = 2 * np.abs(I_fft[:N//2])  # Factor 2 para espectro unilateral
plt.plot(freq_positive, spectrum_positive)
plt.xlim(0, 1000)
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud')
plt.title('Espectro de la Corriente')
plt.grid(True)

# Detalle de armónicos
plt.subplot(2, 2, 3)
harmonic_freqs = [f0 * n for n in range(1, 16)]
harmonic_amps = []
for n in range(1, 16):
    harmonic_freq = n * f0
    idx = np.where((f >= harmonic_freq - tolerance) & (f <= harmonic_freq + tolerance))[0]
    if len(idx) > 0:
        harmonic_amps.append(np.abs(I_fft[idx[0]]))
    else:
        harmonic_amps.append(0)

plt.stem(harmonic_freqs, harmonic_amps)
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud')
plt.title('Componentes Armónicos')
plt.grid(True)

# Formas de onda detalladas
plt.subplot(2, 2, 4)
cycles_to_show = 2
samples_per_cycle = int(fs / f0)
samples_to_show = cycles_to_show * samples_per_cycle
plt.plot(t[:samples_to_show] * 1000, Vrect[:samples_to_show], 'b-', label='Vrect')
plt.plot(t[:samples_to_show] * 1000, I_RC[:samples_to_show] * 100, 'r-', label='I_RC × 100')
plt.xlabel('Tiempo (ms)')
plt.ylabel('Amplitud')
plt.title('2 Ciclos de las Señales')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Función para analizar diferentes configuraciones
def analizar_configuracion(R_val, C_val, descripcion):
    # Simular con nuevos parámetros
    I_RC_temp = np.zeros(len(t))
    Vc_temp = np.zeros(len(t))

    for i in range(1, len(t)):
        dt = t[i] - t[i-1]
        I_RC_temp[i] = (Vrect[i] - Vc_temp[i-1]) / R_val
        Vc_temp[i] = Vc_temp[i-1] + (I_RC_temp[i] * dt) / C_val
        if Vc_temp[i] < 0:
            Vc_temp[i] = 0

    # Calcular THD y FP
    I_fft_temp = np.fft.fft(I_RC_temp) / N
    V1_temp = np.abs(I_fft_temp[fund_idx])

    Vh_squared_temp = 0
    for idx in harmonic_indices:
        if idx < len(I_fft_temp):
            Vh_squared_temp += np.abs(I_fft_temp[idx])**2

    Vh_rms_temp = np.sqrt(Vh_squared_temp)
    THD_temp = (Vh_rms_temp / V1_temp) * 100

    P_activa_temp = np.mean(Vrect * I_RC_temp)
    I_rms_temp = np.sqrt(np.mean(I_RC_temp**2))
    FP_temp = P_activa_temp / (V_rms * I_rms_temp)

    print(f"\n{descripcion}:")
    print(f"  R = {R_val} Ω, C = {C_val*1e6} μF")
    print(f"  THD = {THD_temp:.2f}%")
    print(f"  FP = {FP_temp:.3f}")

# Probar diferentes configuraciones
print("\n" + "=" * 50)
print("COMPARACIÓN DE CONFIGURACIONES")
print("=" * 50)
analizar_configuracion(100, 10e-6, "Carga RC pequeña")
analizar_configuracion(100, 100e-6, "Carga RC media")
analizar_configuracion(100, 1000e-6, "Carga RC grande")
analizar_configuracion(50, 100e-6, "Carga RC R baja")
analizar_configuracion(200, 100e-6, "Carga RC R alta")